In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import os
import cv2
import joblib

# Grid dimensions
GRID_HEIGHT = 6
GRID_WIDTH = 6

# Cell types
EMPTY = 0
OBSTACLE = 1
PICKUP = 2
DROPOFF = 3
ROBOT = 4

# Actions
ACTIONS = ['up', 'down', 'left', 'right']
ACTION_IDX = {a: i for i, a in enumerate(ACTIONS)}

# Rewards
REWARD_PICKUP = 10
REWARD_DROPOFF = 20
REWARD_STEP = -1
REWARD_INVALID = -5
REWARD_OBSTACLE = -10

# Q-learning parameters
alpha = 0.1  # learning rate
gamma = 0.9  # discount factor
epsilon = 0.1  # exploration rate
episodes = 500

# Create folder to save frames
frame_dir = "training_frames"
os.makedirs(frame_dir, exist_ok=True)

# Environment setup
def create_grid():
    grid = np.zeros((GRID_HEIGHT, GRID_WIDTH), dtype=int)
    grid[1, 1] = OBSTACLE
    grid[2, 3] = OBSTACLE
    grid[3, 1] = OBSTACLE
    grid[1, 4] = OBSTACLE
    grid[0, 5] = PICKUP
    grid[5, 0] = DROPOFF
    return grid

def is_valid(y, x):
    return 0 <= y < GRID_HEIGHT and 0 <= x < GRID_WIDTH

def move(y, x, action):
    if action == 'up': y -= 1
    elif action == 'down': y += 1
    elif action == 'left': x -= 1
    elif action == 'right': x += 1
    return y, x

# Initialize Q-table
q_table = np.zeros((GRID_HEIGHT, GRID_WIDTH, 2, len(ACTIONS)))

# Visualization
cmap = mcolors.ListedColormap(['white', 'black', 'green', 'red', 'blue'])

def render(grid, path, final_pos, carried_once, episode):
    vis_grid = grid.copy()
    y, x = final_pos
    vis_grid[y, x] = ROBOT

    plt.imshow(vis_grid, cmap=cmap, vmin=0, vmax=4)

    # Mark start point
    start_y, start_x = path[0]
    plt.plot(start_x, start_y, marker='o', color='purple', markersize=8, label='Start')

    # Add small arrows for path directions
    for i in range(len(path) - 1):
        y1, x1 = path[i]
        y2, x2 = path[i + 1]
        dx = x2 - x1
        dy = y2 - y1
        plt.arrow(x1, y1, dx * 0.3, dy * 0.3, head_width=0.15, head_length=0.15, fc='black', ec='black')

    plt.title(f"Episode {episode} | Final: ({y},{x}) | Carrying: {'Yes' if carried_once else 'No'}")
    frame_path = os.path.join(frame_dir, f"frame_{episode:03d}.png")
    plt.savefig(frame_path)
    plt.clf()

# Training loop
for episode in range(episodes):
    grid = create_grid()
    robot_y, robot_x = 0, 0
    carrying = 0
    carried_once = False
    done = False
    path = [(robot_y, robot_x)]

    for step in range(100):
        state = (robot_y, robot_x, carrying)

        # ε-greedy policy
        if random.uniform(0, 1) < epsilon:
            action = random.choice(ACTIONS)
        else:
            action_idx = np.argmax(q_table[robot_y, robot_x, carrying])
            action = ACTIONS[action_idx]

        new_y, new_x = move(robot_y, robot_x, action)
        reward = REWARD_STEP
        done = False

        if not is_valid(new_y, new_x):
            reward = REWARD_INVALID
            new_y, new_x = robot_y, robot_x
        elif grid[new_y, new_x] == OBSTACLE:
            reward = REWARD_OBSTACLE
            new_y, new_x = robot_y, robot_x
        elif grid[new_y, new_x] == PICKUP and carrying == 0:
            carrying = 1
            carried_once = True
            reward = REWARD_PICKUP
        elif grid[new_y, new_x] == DROPOFF and carrying == 1:
            carrying = 0
            reward = REWARD_DROPOFF
            done = True

        next_state = (new_y, new_x, carrying)
        a_idx = ACTION_IDX[action]

        # Q-learning update
        max_future_q = np.max(q_table[new_y, new_x, carrying])
        q_table[robot_y, robot_x, state[2], a_idx] += alpha * (
            reward + gamma * max_future_q - q_table[robot_y, robot_x, state[2], a_idx]
        )

        robot_y, robot_x = new_y, new_x
        path.append((robot_y, robot_x))

        if done:
            break

    x = int(60 * (episode + 1) / episodes)
    percent = int(100 * (episode + 1) / episodes)
    sys.stdout.write(f"\rProgress: [{'#' * x}{'.' * (60 - x)}]  {percent}% | {episode+1}/{episodes}")
    sys.stdout.flush()
    # Save image after episode
    if episode % 3 == 0 or episode == episodes - 1:
        render(grid, path, (robot_y, robot_x), carried_once, episode)

plt.close()







# Save Q-table for testing
np.save("q_table.npy", q_table)
print("\nTraining completed and q-table saved as 'q_table.npy'")


Progress: [############################################################]  100% | 500/500
Training completed and q-table saved as 'q_table.npy'


In [17]:
# Compile images into video
frame_files = sorted([os.path.join(frame_dir, f) for f in os.listdir(frame_dir) if f.endswith('.png')])

# Read the first image to get frame size
frame = cv2.imread(frame_files[0])
height, width, layers = frame.shape
size = (width, height)
image_count = len(frame_files)

# Create VideoWriter object
out = cv2.VideoWriter('training_video.avi', cv2.VideoWriter_fourcc(*'XVID'), 3, size)

for i, filename in enumerate(frame_files):
    frame = cv2.imread(filename)
    out.write(frame)

    x = int(60 * (i + 1) / image_count)
    percent = int(100 * (i + 1) / image_count)
    sys.stdout.write(f"\rProgress: [{'#' * x}{'.' * (60 - x)}]  {percent}% | {i+1}/{image_count}")
    sys.stdout.flush()
    
    # Show every 5th frame for preview
    if i % 5 == 0:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    

# Release video file
out.release()
print("\nVideo created successfully!")

Progress: [############################################################]  100% | 168/168
Video created successfully!


In [11]:
len(frame_files)

168